# Gwent Pro Rank analysis

First we'll need to get the data from the playgwent website. 


In [2]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np

seasons = [
    ('M2_01 Wolf 2020', 'https://masters.playgwent.com/en/rankings/masters-2/season-of-the-wolf/1/{page}', './output/season_of_the_wolf_2020.xlsx'),
    ('M2_02 Love 2020', 'https://masters.playgwent.com/en/rankings/masters-2/season-of-love/1/{page}', './output/season_of_love_2020.xlsx'),
    ('M2_03 Bear 2020', 'https://masters.playgwent.com/en/rankings/masters-2/season-of-the-bear/1/{page}', './output/season_of_the_bear_2020.xlsx'),
    ('M2_04 Elf 2020', 'https://masters.playgwent.com/en/rankings/masters-2/season-of-the-elf/1/{page}', './output/season_of_the_elf_2020.xlsx'),
    ('M2_05 Viper 2020', 'https://masters.playgwent.com/en/rankings/masters-2/season-of-the-viper/1/{page}', './output/season_of_the_viper_2020.xlsx'),
    ('M2_06 Magic 2020', 'https://masters.playgwent.com/en/rankings/masters-2/season-of-the-magic/1/{page}', './output/season_of_magic_2020.xlsx'),
    ('M2_07 Griffin 2020', 'https://masters.playgwent.com/en/rankings/masters-2/season-of-the-griffin/1/{page}', './output/season_of_the_griffin_2020.xlsx'),
    ('M2_08 Draconic 2020', 'https://masters.playgwent.com/en/rankings/masters-2/season-of-the-draconid/1/{page}', './output/season_of_the_draconid_2020.xlsx')
]

dataframes = []

for season, url_template, output_path in seasons:
    if os.path.exists(output_path):
        df = pd.read_excel(output_path).drop(['Unnamed: 0'], axis=1)
    else:
        output = []

        for i in tqdm(range(1, 150)):
            url = url_template.replace('{page}', str(i))
            try:
                r = requests.get(url)
                soup = BeautifulSoup(r.text, 'html.parser')

                rows = soup.findAll("div", {"class": "c-ranking-table__tr"})
                for row in rows:
                    flag = row.find("i", {"class": "flag-icon"})["class"][1]
                    new_record = {
                        'rank': row.find("div", {"class": "td-number"}).text.strip(),
                        'name': row.find("div", {"class": "td-nick"}).text.strip(),
                        'country': flag.replace('flag-icon-', '').upper(),
                        'matches': int(row.find("div", {"class": "td-matches"}).text.strip().replace(' matches', '')),
                        'mmr': int(row.find("div", {"class": "td-mmr"}).text.strip().replace(',', '')),
                        'season': season
                    }
                    output.append(new_record)
            except:
                pass


        df = pd.DataFrame(output).drop_duplicates()
        df.to_excel(output_path)
    dataframes.append(df)
    
# Concatenate dataframes and drop any records with missing values
full_df = pd.concat(dataframes).dropna()

# Human readable countries

We have the 2 letter code for the country of each participant, let's use the pycountry library to convert that to the full name.

In [3]:
import pycountry

def convert_to_full(two_letter_code):
    try:
        out = pycountry.countries.get(alpha_2=two_letter_code).name.split(',')[0]
    except:
        print(f'could not convert {two_letter_code}')
        out = two_letter_code
    return out

full_df['country'] = full_df['country'].apply(lambda x: convert_to_full(x))

# National rank

Apart from the global rank, we'll add a participant's national rank as well.

In [4]:
full_df['national_rank'] = full_df.groupby(['country','season'])["mmr"].rank("dense", ascending=False)

# Ladder Efficiency

While getting to a higher MMR is the main goal of ladder. There is an argument to be made that the number of games a participant plays to reach a certain point should not be overlooked. Someone that played fewer games to get to a certain MMR climbed the ladder in a more efficient way. There are two metrics that can be used, we take the MMR of a player, substract the minimal MMR, which is 9600, and divide that by the number of matches played. Though, as the higher up you are in ladder, the more fierce the competition becomes, to take this into account Lerio2 from Team Legacy proposed to divide by the square-root of the number of matches instead.

One additional consideration is that players that were in the top 500 the previous season didn't have to climb back up to pro rank. Both metrics actually give them a major advantage

In [5]:
full_df['efficiency'] = ((full_df['mmr']-9600))/full_df['matches']
full_df['lei'] = ((full_df['mmr']-9600))/np.sqrt(full_df['matches'])

In [6]:
full_df.head()

,rank,name,country,matches,mmr,season,national_rank,efficiency,lei
0,1,kolemoen,Germany,431,10484,M2_01 Wolf 2020,1.0,2.051044,42.580782
1,2,kams134,Poland,923,10477,M2_01 Wolf 2020,1.0,0.950163,28.866807
2,3,TailBot,Poland,538,10472,M2_01 Wolf 2020,2.0,1.620818,37.594590
3,4,Pajabol,Poland,820,10471,M2_01 Wolf 2020,3.0,1.062195,30.416639
4,5,Adzikov,Poland,1105,10442,M2_01 Wolf 2020,4.0,0.761991,25.329753


# Get population for each country

To calculate the number of pro players per capita we'll need the number of inhabitants per country. On Kaggle a dataset was available for 2019. As this doesn't differ much from one year to another (and those will only result into changes several digits behind the decimal point it) the 2019 list is enough for our purpose. The code below loads the data, and generates country names from the two character ISO code using the same package as used before to avoid mismatches.


In [7]:
# data from kaggle https://www.kaggle.com/erikbruin/countries-of-the-world-iso-codes-and-population/data#
# the number are indicated very strangely in the csv with a decimal point and need to be multiplied with 1000

population_df = pd.read_csv('./data/countries_by_population_2019.csv')[['name', 'pop2019']]
country_codes = pd.read_csv('./data/country_codes_2020.csv')[['name', 'cca2']]
population_df = pd.merge(population_df, country_codes, how='left', left_on='name', right_on='name')

population_df['country'] = population_df['cca2'].apply(lambda x: convert_to_full(x))
population_df['pop2019'] = (population_df['pop2019'] * 1000).astype(int)

could not convert nan


In [8]:
per_season_df = full_df.groupby(['season']).agg(
    min_mmr     = pd.NamedAgg('mmr', 'min'),
    max_mmr     = pd.NamedAgg('mmr', 'max'),
    num_matches = pd.NamedAgg('matches', 'sum')
).reset_index()
per_season_df

,season,min_mmr,max_mmr,num_matches
0,M2_01 Wolf 2020,2407,10484,699496
1,M2_02 Love 2020,7776,10537,769358
2,M2_03 Bear 2020,9427,10669,862678
3,M2_04 Elf 2020,9666,10751,1004830
4,M2_05 Viper 2020,9635,10622,859640
5,M2_06 Magic 2020,9624,10597,793401
6,M2_07 Griffin 2020,9698,10667,996742
7,M2_08 Draconic 2020,9622,10454,744260


In [9]:
# Lets do some counts per season and country
national_stats = full_df.groupby(['season','country']).agg(
    total_matches = pd.NamedAgg('matches', 'sum'),
    num_players = pd.NamedAgg('name', 'count')
).reset_index()

# Add the population (from 2019) to each country and calculate the number of pro players (top 2860) per million inhabitants
national_stats = pd.merge(national_stats, population_df.drop(['name', 'cca2'], axis=1), on='country', how='left')
national_stats['pro_players_per_million'] = (national_stats['num_players']*1000000)/national_stats['pop2019']
national_stats['matches_per_player'] = national_stats['total_matches']/national_stats['num_players']

national_stats

,season,country,total_matches,num_players,pop2019,pro_players_per_million,matches_per_player
0,M2_01 Wolf 2020,Algeria,250,1,43053054.0,0.023227,250.000000
1,M2_01 Wolf 2020,Argentina,1480,4,44780677.0,0.089324,370.000000
2,M2_01 Wolf 2020,Armenia,546,1,2957731.0,0.338097,546.000000
3,M2_01 Wolf 2020,Australia,6193,27,25203198.0,1.071293,229.370370
4,M2_01 Wolf 2020,Austria,3981,16,8955102.0,1.786691,248.812500
...,...,...,...,...,...,...,...
690,M2_08 Draconic 2020,United States,52064,198,329064917.0,0.601705,262.949495
691,M2_08 Draconic 2020,Uzbekistan,1366,5,32981716.0,0.151599,273.200000
692,M2_08 Draconic 2020,Venezuela,507,2,28515829.0,0.070136,253.500000
693,M2_08 Draconic 2020,Viet Nam,3406,12,96462106.0,0.124401,283.833333


# Creating a ranking of nations

We'll select countries with three or more playing in the pro rank, select the top 3 players and calculate the average MMR and total MMR for the top
3 players of the country.



In [10]:
num_player_included = 3   # Here we define the number of players in Pro Ladder to be considered

players_per_country = full_df.groupby(['season','country']).agg(
    num_players = pd.NamedAgg('name', 'count')
).reset_index()

players_per_country = players_per_country[players_per_country['num_players'] >= num_player_included]

top_per_country = pd.merge(players_per_country[['season', 'country']], full_df, how='inner', on=['season', 'country'])
top_per_country = top_per_country[top_per_country['national_rank'] <= num_player_included]
top_per_country

,season,country,rank,name,matches,mmr,national_rank,efficiency,lei
0,M2_01 Wolf 2020,Argentina,174,Srchenko,393,9887,1.0,0.730280,14.477235
1,M2_01 Wolf 2020,Argentina,471,eltroco,339,9758,2.0,0.466077,8.581382
2,M2_01 Wolf 2020,Argentina,846,MorgannFreemann,208,9660,3.0,0.288462,4.160251
4,M2_01 Wolf 2020,Australia,135,whitecool,404,9914,1.0,0.777228,15.622084
5,M2_01 Wolf 2020,Australia,211,Pacifier3750,453,9863,2.0,0.580574,12.356818
...,...,...,...,...,...,...,...,...,...
22575,M2_08 Draconic 2020,Uzbekistan,694,StormMedia,308,9817,2.0,0.704545,12.364723
22576,M2_08 Draconic 2020,Uzbekistan,1018,sarvar97,241,9758,3.0,0.655602,10.177675
22579,M2_08 Draconic 2020,Viet Nam,189,umeu,634,10000,1.0,0.630915,15.886029
22580,M2_08 Draconic 2020,Viet Nam,569,ductruongyo,222,9844,2.0,1.099099,16.376208


In [11]:
top_stats = top_per_country.groupby(['season','country']).agg(
    mean_mmr                = pd.NamedAgg('mmr', 'mean'),
    total_mmr               = pd.NamedAgg('mmr', 'sum'),
    mean_matches_per_player = pd.NamedAgg('matches', 'mean'),
    total_matches           = pd.NamedAgg('matches', 'sum')
).reset_index()

# Here we add the same ranking and effeciency statistics for national teams as for individual players
top_stats['national_rank'] = top_stats.groupby(['season'])["mean_mmr"].rank("dense", ascending=False)
top_stats['efficiency'] = ((top_stats['mean_mmr']-9600))/top_stats['mean_matches_per_player']
top_stats['lei'] = ((top_stats['mean_mmr']-9600))/np.sqrt(top_stats['mean_matches_per_player'])

top_stats

,season,country,mean_mmr,total_mmr,mean_matches_per_player,total_matches,national_rank,efficiency,lei
0,M2_01 Wolf 2020,Argentina,9768.333333,29305,313.333333,940,34.0,0.537234,9.509700
1,M2_01 Wolf 2020,Australia,9869.666667,29609,336.333333,1009,22.0,0.801784,14.704231
2,M2_01 Wolf 2020,Austria,9905.666667,29717,392.333333,1177,19.0,0.779099,15.431938
3,M2_01 Wolf 2020,Belarus,10064.333333,30193,390.000000,1170,11.0,1.190598,23.512432
4,M2_01 Wolf 2020,Belgium,9901.000000,29703,415.666667,1247,21.0,0.724138,14.763655
...,...,...,...,...,...,...,...,...,...
479,M2_08 Draconic 2020,Ukraine,10230.666667,30692,417.000000,1251,5.0,1.512390,30.883880
480,M2_08 Draconic 2020,United Kingdom,10152.666667,30458,330.000000,990,8.0,1.674747,30.423299
481,M2_08 Draconic 2020,United States,10169.666667,30509,509.666667,1529,7.0,1.117724,25.233512
482,M2_08 Draconic 2020,Uzbekistan,9834.333333,29503,246.666667,740,45.0,0.950000,14.920344


# Export everything

Lets exports all data to Excel files so they can be further analysed without pandas/python. 

In [16]:
full_df.to_excel('./output/player_stats.xlsx')
national_stats.to_excel('./output/national_stats.xlsx')

# Some quick checks about Belgian players

Why Belgium ?! Simple I'm from there, and I had to pick a country to have a look at to see if everything holds up. 

In [13]:
full_df[full_df['country'] == 'Belgium']

,rank,name,country,matches,mmr,season,national_rank,efficiency,lei
8,9,alessio1996,Belgium,682,10330,M2_01 Wolf 2020,1.0,1.070381,27.953145
664,665,WellanXM,Belgium,369,9696,M2_01 Wolf 2020,2.0,0.260163,4.997560
747,748,mppqlmd,Belgium,196,9677,M2_01 Wolf 2020,3.0,0.392857,5.500000
1146,1147,selderslaghspj,Belgium,191,9617,M2_01 Wolf 2020,4.0,0.089005,1.230077
1235,1236,CD318,Belgium,325,9608,M2_01 Wolf 2020,5.0,0.024615,0.443760
1891,1892,FractlZero,Belgium,157,7216,M2_01 Wolf 2020,6.0,-15.184713,-190.263913
2456,2457,NotsoGodlyDog,Belgium,71,4665,M2_01 Wolf 2020,7.0,-69.507042,-585.676748
2553,2554,Haze1083,Belgium,137,4050,M2_01 Wolf 2020,8.0,-40.510949,-474.168500
2647,2648,_original,Belgium,156,3412,M2_01 Wolf 2020,9.0,-39.666667,-495.436508
361,362,NotsoGodlyDog,Belgium,153,9860,M2_02 Love 2020,1.0,1.699346,21.019754


In [14]:
top_stats[top_stats['country'] == 'Belgium']

,season,country,mean_mmr,total_mmr,mean_matches_per_player,total_matches,national_rank,efficiency,lei
4,M2_01 Wolf 2020,Belgium,9901.000000,29703,415.666667,1247,21.0,0.724138,14.763655
65,M2_02 Love 2020,Belgium,9774.333333,29323,175.666667,527,40.0,0.992410,13.153331
183,M2_04 Elf 2020,Belgium,9713.000000,29139,452.666667,1358,57.0,0.249632,5.311158
241,M2_05 Viper 2020,Belgium,9807.333333,29422,441.000000,1323,54.0,0.470144,9.873016
304,M2_06 Magic 2020,Belgium,9757.666667,29273,250.333333,751,55.0,0.629827,9.965074
365,M2_07 Griffin 2020,Belgium,9878.666667,29636,276.666667,830,53.0,1.007229,16.753541
426,M2_08 Draconic 2020,Belgium,9844.000000,29532,192.666667,578,42.0,1.266436,17.578691


In [15]:
full_df[full_df['name'] == 'sepro']

,rank,name,country,matches,mmr,season,national_rank,efficiency,lei
1137,1138,sepro,Belgium,254,9746,M2_06 Magic 2020,2.0,0.574803,9.160855
1166,1167,sepro,Belgium,260,9820,M2_07 Griffin 2020,2.0,0.846154,13.643821
1869,1870,sepro,Belgium,136,9677,M2_08 Draconic 2020,7.0,0.566176,6.602696
